# Test locking on load

In [ ]:
!lamin login testuser2

In [ ]:
import pytest
import lamindb_setup as ln_setup
from lamindb_setup.dev.cloud_sqlite_locker import Locker
from lamindb_setup.dev.upath import UPath

In [ ]:
root = UPath("s3://lndb-setup-ci", cache_regions=True)

In [ ]:
exclusion = root / "exclusion/lndb-load-test-lock"
if exclusion.exists():
    exclusion.rmdir()

In [ ]:
assert ln_setup.settings.user.handle == "testuser2"

In [ ]:
ln_setup.init(storage="s3://lndb-setup-ci", name="lndb-load-test-lock")
ln_setup.close()

In [ ]:
# lock with some random id
locker = Locker("randuseridtt", storage_root=root, name="lndb-load-test-lock")

In [ ]:
locker.lock()

In [ ]:
with pytest.raises(RuntimeError):
    ln_setup.load("lndb-load-test-lock")

In [ ]:
locker.unlock()

In [ ]:
ln_setup.load("lndb-load-test-lock")
ln_setup.close()

In [ ]:
# lock through testuser1, who has user id "DzTjkKse"
locker = Locker("DzTjkKse", storage_root=root, name="lndb-load-test-lock")

In [ ]:
locker.lock()

In [ ]:
with pytest.raises(RuntimeError) as error:
    ln_setup.load("lndb-load-test-lock")

assert (
    error.exconly()
    == "RuntimeError: Cannot load the instance, it is locked by 'testuser1' (id:"
    " 'DzTjkKse', name: 'Test User1')."
)

In [ ]:
locker.unlock()

In [ ]:
ln_setup.load("lndb-load-test-lock")

In [ ]:
ln_setup.delete("lndb-load-test-lock", force=True)